In [12]:
import sys
sys.path.insert(0, '/home/sos/Dialogue_Research/color_in_context/system/dialogue_manager')
from dataset.cic import make_or_load_cic
import string
import numpy as np
import dataset
import pickle
import os

In [2]:
cic = make_or_load_cic()

In [57]:
# cic._df[(cic._df['split'] == 'train') & (cic._df['lux_difficulty_rating'].sin([0,1,2,3]))].groupby('condition').count()
# cic._df[cic._df['split'] == 'dev'].iloc[:2500].groupby('condition').count()

y = cic._df[cic._df['condition'] == 'close'].sample().index
y[0]
# x = [0 for i in range(y + 1)]
# x[y]

8918

In [3]:
vocabulary = cic._color_vocab

tokens = set()
for i in range(len(vocabulary)):
    color_term = vocabulary.lookup_index(i)
    for token in color_term.split():
        for term in token.split('-'):
            tokens.add(term)
            
def expansion(word):
    result = set()
    #deletions
    #assuming mistake in upto one missing characters
    deletions = set()
    for i in range(len(word)):
        new_word = word[:i] + word[i+1:]
        deletions.add(new_word)
    #additions
    for letter in string.ascii_lowercase:
        for i in range(len(word) + 1):
            new_word = word[:i] + letter + word[i:]
            result.add(new_word)
    #deletion addition
    for letter in string.ascii_lowercase:
        for word in deletions:
            for i in range(len(word) + 1):
                new_word = word[:i] + letter + word[i:]
                result.add(new_word)
    
    
    return result | deletions

cic._df.iloc[[cic[0]['row_index']]]
token_counts = dict()
for item in cic:
    color_term = cic._df.iloc[item['row_index']]['lux_label']
    for token in color_term.split():
        for term in token.split('-'):
            token_counts.setdefault(term, 0)
            token_counts[term] += 1

# expansion('blue')
def expansion_to_correct_map(tokens, token_counts):
    expansions = dict()
    for term in tokens:
        modifications = expansion(term)
        for modified in modifications:
            expansions.setdefault(modified, set())
            try:
                expansions[modified].add((term, token_counts[term]))
            except Exception as e:
                expansions[modified].add((term, 0))
    return expansions


In [14]:
exp_correct_map = expansion_to_correct_map(tokens, token_counts)
pickle.dump(exp_correct_map, open(os.path.dirname(dataset.__file__) + '/spellcheck_map', 'wb'))

In [6]:
def get_correct_word(word, exp_correct_map):
    possible_set = list(exp_correct_map[word])
    if word in [term[0] for term in possible_set]:
        return word
    else:
        ix = np.argmax([term[1] for term in possible_set])
        return possible_set[ix][0]
    
for word in ['blus', 'brown', 'reddish', 'yellwo', 'yllow', 'blu']:
    print(get_correct_word(word, exp_correct_map))

blue
brown
reddish
yellow
yellow
blue
